## more advanced aspects

### time resolved MS
- peak storage / data storage
- 2D storage
    - hdf5
    - compression
- space charge effect
- recalibration

### 2D
- basic principle
    - sampling, Nyquist, processing 
- but
    - $S(t_1) \propto \sqrt{1+\cos(\omega_1 t_1)}$
    - Maria puzzle
    - FT of $S(t_1)$
        - harmonics
        - fragmentation optimization
    - freq generator continuous phase 
- difference in F1 and F2
    - Nyquists
    - band limited
- causality in 2D
    - hypercomplex algebra.
- phasing
    - F2 is 2nd order
    - F1 is 1st order
- NUS
    - FISTA -we've seen it

-->